<a href="https://colab.research.google.com/github/csabiu/Astrostatistics/blob/main/Nested_Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install the nested sampler

In [ ]:
pip install -U dynesty

In [ ]:
import dynesty

In [ ]:
from astropy.cosmology import LambdaCDM, FlatLambdaCDM
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Math, Latex

In [ ]:
cosmo1=FlatLambdaCDM(H0=70, Om0=0.3)
z=np.arange(0.1,1,0.1)
da_data=cosmo1.angular_diameter_distance(z).value
da_err=0.05*da_data

plt.errorbar(z,da_data,yerr=da_err,fmt='.')
plt.xlabel('redshift, z')
plt.ylabel(r'$D_A$ [Mpc]')

In [ ]:
def prior_transform(theta):
    """
    A function defining the tranform between the parameterisation in the unit hypercube
    to the true parameters.
    Args:
        theta (tuple): a tuple containing the parameters.

    Returns:
        tuple: a new tuple or array with the transformed parameters.
    """

    Om, H0 = theta # unpack the parameters (in their unit hypercube form)

    Ommin = 0.1  # lower bound on uniform prior
    Ommax = 0.5   # upper bound on uniform prior

    H0min = 50.
    H0max = 100.

    H0 = H0*(H0max-H0min) + H0min  # convert back to real value
    Om = Om*(Ommax-Ommin) + Ommin

    return (Om, H0)

In [ ]:
def loglikelihood_dynesty(theta):
    """
    The log-likelihood function.
    """

    Om, H0 = theta # unpack the parameters

    cosmo_Om_p=FlatLambdaCDM(H0=H0, Om0=Om)
    da_model_Om_p=cosmo_Om_p.angular_diameter_distance(z).value
    chisq=np.sum((da_data-da_model_Om_p)**2/da_err**2)

    return -0.5 * chisq

# Lets run the sampler!

In [ ]:
nlive = 100      # number of live points
bound = 'single'   # use MutliNest algorithm for bounds
ndims = 2         # two parameters
sample = 'unif'   # uniform sampling
tol = 0.1         # the stopping criterion

from dynesty import NestedSampler

sampler = NestedSampler(loglikelihood_dynesty, prior_transform, ndims,
                        bound=bound, sample=sample, nlive=nlive)

sampler.run_nested(dlogz=tol, print_progress=True) # don't output progress bar




# Lets get the evidence!

In [ ]:
res = sampler.results # get results dictionary from sampler

logZdynesty = res.logz[-1]        # value of logZ
logZerrdynesty = res.logzerr[-1]  # estimate of the statistcal uncertainty on logZ

print("log(Z) = {} ± {}".format(logZdynesty, logZerrdynesty))

In [ ]:
plt.scatter(res.samples[:,0],res.samples[:,1])

# Lets draw 2D contours as before

In [ ]:
# get function that resamples from the nested samples to give sampler with equal weight
from dynesty.utils import resample_equal

# draw posterior samples
weights = np.exp(res['logwt'] - res['logz'][-1])
samples_dynesty = resample_equal(res.samples, weights)

In [ ]:
pip install corner

In [ ]:
import corner

fig = corner.corner(
    samples_dynesty, labels=['$\Omega_M$','$H_0$'], truths=[0.3,70.], smooth=1.,
);

# Lets print or marginalised 1D results with errorbars

In [ ]:
from IPython.display import display, Math
labelss=['\Omega_M','H_0']
for i in range(2):
    mcmc = np.percentile(samples_dynesty[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    txt = "\mathrm{{{3}}} = {0:.3f}_{{-{1:.3f}}}^{{{2:.3f}}}"
    txt = txt.format(mcmc[1], q[0], q[1], labelss[i])
    display(Math(txt))